---
title: "DAP II Problem Set 5"
author: Sara Van Valkenburgh and Jennifer Edouard
date: November 9, 2025
geometry: 
    margin=0.75in
fontsize: 10pt
format: 
  pdf:
    include-in-header: 
       text: |
         \usepackage{fvextra}
         \DefineVerbatimEnvironment{Highlighting}{Verbatim}{breaklines,commandchars=\\\{\}}
include-before-body:
  text: |
    \RecustomVerbatimEnvironment{verbatim}{Verbatim}{
      showspaces = false,
      showtabs = false,
      breaksymbolleft={},
      breaklines
    }
output:
  echo: false
  eval: false
---


**Due 11/9 at 5:00PM Central. Worth 100 points + 10 points extra credit.**

## Submission Steps (10 pts)
1. This problem set is a paired problem set.
2. Play paper, scissors, rock to determine who goes first. Call that person *Partner 1*.
    - Partner 1 (name and cnet ID): Sara Van Valkenburgh, vanvals
    - Partner 2 (name and cnet ID): Jennifer Edouard, jkedouard
3. Partner 1 will accept the `ps5` and then share the link it creates with their partner. You can only share it with one partner so you will not be able to change it after your partner has accepted. 
4. "This submission is our work alone and complies with the 30538 integrity policy." Add your initials to indicate your agreement: \*\*sv\*\* \*\*je\*\*
5. "I have uploaded the names of anyone else other than my partner and I worked with on the problem set **[here](https://docs.google.com/forms/d/185usrCREQaUbvAXpWhChkjghdGgmAZXA3lPWpXLLsts/edit)**"  (1 point)
6. Late coins used this pset: \*\*\_\_\*\* Late coins left after submission: \*\*\_\_\*\*
7. Knit your `ps5.qmd` to an PDF file to make `ps5.pdf`, 
    * The PDF should not be more than 25 pages. Use `head()` and re-size figures when appropriate. 
8. (Partner 1): push  `ps5.qmd` and `ps5.pdf` to your github repo.
9. (Partner 1): submit `ps5.pdf` via Gradescope. Add your partner on Gradescope.
10. (Partner 1): tag your submission in Gradescope

\newpage


In [ ]:
import pandas as pd
import altair as alt
import time
import requests
from bs4 import BeautifulSoup

import warnings 
warnings.filterwarnings('ignore')
alt.renderers.enable("png")

## Step 1: Develop initial scraper and crawler

### 1. Scraping (PARTNER 1)


In [ ]:
url = 'https://oig.hhs.gov/fraud/enforcement/'
response = requests.get(url)

soup = BeautifulSoup(response.text, 'html.parser')

In [ ]:
# Empty lists to store the information
titles = []
dates = []
categories = []
links = []

In [ ]:
# Title of the enforcement action
for heading in soup.find_all('h2', class_='usa-card__heading'):
    title = heading.find('a').get_text(strip=True)
    titles.append(title)

In [ ]:
# Date
for date_tag in soup.find_all('span', class_='text-base-dark padding-right-105'):
    date = date_tag.get_text(strip=True)
    dates.append(date)

In [ ]:
# Category
for ul in soup.find_all('ul', class_='display-inline add-list-reset'):
    for li in ul.find_all('li'):
        category = li.get_text(strip=True)
        categories.append(category)

In [ ]:
# Link associated with enforcement action
for action in soup.find_all('a', href=True):
    href = action['href']
    if href.startswith("/fraud/enforcement/"):
        full_link = f"https://oig.hhs.gov{href}"
        links.append(full_link)

# Remove the first three links; they are irrelevant
links = links[3:] 

In [ ]:
#Put into a dataframe
data = {
    'Title': titles,
    'Date': dates,
    'Category': categories,
    'Link': links
}
df= pd.DataFrame(data)
df.head()

### 2. Crawling (PARTNER 1)


In [ ]:
def get_agency_name(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    agency_name = "N/A"
    agency_label = soup.find(
        'span', class_='padding-right-2 text-base', string="Agency:")

    if agency_label and agency_label.next_sibling:
        agency_name = agency_label.next_sibling.get_text(strip=True)

    return agency_name

In [ ]:
data = {
    'Title': titles,
    'Date': dates,
    'Category': categories,
    'Link': links
}
df = pd.DataFrame(data)
df['Agency'] = None

for idx, link in enumerate(df['Link']):
    df.at[idx, 'Agency'] = get_agency_name(link)

df.head()

Source: https://stackoverflow.com/questions/16476924/how-can-i-iterate-over-rows-in-a-pandas-dataframe

## Step 2: Making the scraper dynamic

### 1. Turning the scraper into a function 

* a. Pseudo-Code (PARTNER 2)

Define function 
    - If the input year is less than 2013:
        - Display message: "Please enter a year greater than or equal to 2013."
        - Exit 

    - Set the base URL for enforcement actions
    - Create an empty DataFrame to store all actions data

    - Define the start date 
    - Define the current date as today's date 

    - Set the initial page number to 1

    - Begin loop to scrape each page
        - URL for the current page
        - Parse page content with BeautifulSoup
        
        - Extract titles
        - Extract dates
            - Convert each date text to a datetime object
	    - Extract categories
        - Extract links
        - Extract agencies

        - Create empty dictionary to store information

        - Loop through each date:
            - If the date is valid:
                - If the date is before the start date,  break the loop
                - If the date is within the range (start_date to current_date), append the title, formatted date, category, and link 

	    - Put into a DataFrame
        - Pause for 1 second to avoid overloading the server
        - Check if a "next" page link exists:
            - If yes, increment the page number and continue to the next page
            - If not, exit the loop

    - Once scraping is complete, save the DataFrame to a CSV file

* b. Create Dynamic Scraper (PARTNER 2)

In [ ]:
from datetime import datetime


def scrape_enforcement_actions(year, month):
    # make it restart if a year lower than 2013 is entered
    if year < 2013:
        print("Please enter a year greater than or equal to 2013.")
        return

    # same url opening for all the pages scraped
    base_url = "https://oig.hhs.gov/fraud/enforcement/"
    all_actions = pd.DataFrame()

    # making sure it stops at today's date (it might do that already bc there's no data past today...)
    start_date = datetime(year, month, 1)
    current_date = datetime.now()

    # establishing the pages so it can go on from there
    page_num = 1

    stop_scraping = False

    # this just uses the code that Partner 1 wrote, but making it crawl through all the pages
    while not stop_scraping:
        page_url = f"{base_url}?page={page_num}"
        print(f"Scraping page: {page_url}")

        response = requests.get(page_url)
        soup = BeautifulSoup(response.text, "html.parser")

        titles = [heading.find('a').get_text(strip=True)
                  for heading in soup.find_all('h2', class_='usa-card__heading')]

        date_texts = [date_tag.get_text(strip=True)
                      for date_tag in soup.find_all('span', class_='text-base-dark padding-right-105')]
        dates = []
        for date_text in date_texts:
            try:
                date = datetime.strptime(date_text, "%B %d, %Y")
                dates.append(date)
                # Check if date is before start_date and set stop_scraping if it is
                if date < start_date:
                    stop_scraping = True
                    break
            except ValueError:
                print(f"Error parsing date: {date_text}")
                dates.append(None)

        if stop_scraping:
            break

        categories = [li.get_text(strip=True)
                      for ul in soup.find_all('ul', class_='display-inline add-list-reset')
                      for li in ul.find_all('li')]

        links = [f"https://oig.hhs.gov{a['href']}"
                 for a in soup.find_all('a', href=True)
                 if a['href'].startswith("/fraud/enforcement/")][3:]

        # filter based on start and end dates
        filtered_actions = {
            "Title": [],
            "Date": [],
            "Category": [],
            "Link": [],
            "Agency": []
        }
        for i in range(len(dates)):
              if dates[i]:
                    if dates[i] < start_date:
                        stop_scraping = True
                        break
                    if start_date <= dates[i] <= current_date:
                        filtered_actions["Title"].append(titles[i])
                        filtered_actions["Date"].append(
                            dates[i].strftime("%Y-%m-%d"))
                        filtered_actions["Category"].append(categories[i])
                        filtered_actions["Link"].append(links[i])
                        agency_name = get_agency_name(links[i])
                        filtered_actions["Agency"].append(agency_name)

        # add filtered actions to the DataFrame
        page_df = pd.DataFrame(filtered_actions)
        all_actions = pd.concat([all_actions, page_df], ignore_index=True)

        # wait before moving to the next page
        time.sleep(1)

        # check for the next page
        next_button = soup.find('a', {'class': 'pagination-next'})
        if next_button:
            page_num += 1
        else:
            break

    # save results to CSV
    file_name = f"enforcement_actions_{year}_{month:02d}.csv"
    all_actions.to_csv(file_name, index=False)
    print(f"Data saved to {file_name}")

In [ ]:
# Testing it for January 2023
scrape_enforcement_actions(2023, 1)

In [ ]:
df2023 = pd.read_csv("data/enforcement_actions_2023_01.csv")
len(df2023)

Number of enforcement actions in final data frame: 1510

Date and details of the earliest enforcement action scraped: Podiatrist Pays $90,000 To Settle False Billing Allegations, 2023-01-03, Criminal and Civil Actions, https://oig.hhs.gov/fraud/enforcement/podiatrist-pays-90000-to-settle-false-billing-allegations/, U.S. Attorney’s Office, Southern District of Texas

* c. Test Partner's Code (PARTNER 1)


In [ ]:
# Testing for January 2021
scrape_enforcement_actions(2021, 1)

In [ ]:
df2021 = pd.read_csv("data/enforcement_actions_2021_01.csv")
len(df2021)

Number of enforcement actions in final data frame: 3016

Date and details of the earliest enforcement action scraped: The United States And Tennessee Resolve Claims With Three Providers For False Claims Act Liability Relating To ‘P-Stim’ Devices For A Total Of $1.72 Million, 2021-01-04, Criminal and Civil Actions, https://oig.hhs.gov/fraud/enforcement/the-united-states-and-tennessee-resolve-claims-with-three-providers-for-false-claims-act-liability-relating-to-p-stim-devices-for-a-total-of-172-million/, U.S. Attorney's Office, Middle District of Tennessee

## Step 3: Plot data based on scraped data

### 1. Plot the number of enforcement actions over time (PARTNER 2)


In [ ]:
# filtering down to just the state agency enforcement actions
filtered_2021_state_agencies = df2021[df2021["Agency"].str.startswith("State of", na=False)]

# aggregating by month and year
filtered_2021_state_agencies["Date"] = pd.to_datetime(filtered_2021_state_agencies["Date"])

filtered_2021_state_agencies["Year"] = filtered_2021_state_agencies["Date"].dt.year
filtered_2021_state_agencies["Month"] = filtered_2021_state_agencies["Date"].dt.month

filtered_2021_state_agencies_aggregate = (
    filtered_2021_state_agencies
    .groupby(["Year", "Month"])
    .size()
    .reset_index(name="Enforcement_Actions_Count")
)

filtered_2021_state_agencies_aggregate["Date"] = pd.to_datetime(
    filtered_2021_state_agencies_aggregate[["Year", "Month"]].assign(Day=1)
)

# making the line plot
line_chart_2021_state_agencies = alt.Chart(filtered_2021_state_agencies_aggregate).mark_line(
    color="hotpink",
    strokeWidth=2
).encode(
    x=alt.X("Date:T", title="Date"),
    y=alt.Y("Enforcement_Actions_Count:Q", title="Number of Enforcement Actions")
).properties(
    title="Monthly State Agency Enforcement Actions Since January 2021",
    width=600,
    height=400
)

line_chart_2021_state_agencies

### 2. Plot the number of enforcement actions categorized: (PARTNER 1)

* based on "Criminal and Civil Actions" vs. "State Enforcement Agencies"

In [ ]:
# make sure date is in the right format for the plots
df2021['Date'] = pd.to_datetime(df2021['Date'])
df2021['Year-Month'] = df2021['Date'].dt.to_period('M').dt.to_timestamp()

In [ ]:
# filter to relevant categories and make a new df that has the counts for each category
filtered_df = df2021[df2021['Category'].isin(['Criminal and Civil Actions', 'State Enforcement Agencies'])]
category_counts = filtered_df.groupby(['Year-Month', 'Category']).size().reset_index(name='Count')

In [ ]:
# make the plot
alt.Chart(category_counts).mark_line().encode(
    x=alt.X('Year-Month:T', 
            title='Month', 
            axis=alt.Axis(
                format='%b %Y',  
                tickCount=12,  
                labelAngle=-45, 
                tickSize=5      
            )),
    y=alt.Y('Count:Q', title='Number of Enforcement Actions'),
    color=alt.Color('Category:N', title='Category'),
).properties(
    title='Total Enforcement Actions Over Time: Criminal and Civil Actions vs. State Enforcement Agencies',
    width = 600
)

* based on five topics

In [ ]:
import re

# filter to only enforcement actions within the category crimial and civil actions
filtered_df2 = df2021.loc[df2021['Category'] == 'Criminal and Civil Actions']

# function to search for keywords and form categories
def classify_topic(title):
    if re.search(r'health|medical|healthcare|doctor|physicians?|care|medicare|medicaid|dentists?|dental|hospitals?|clinics?|dentists?|insurance', title, re.IGNORECASE):
        return 'Health Care Fraud'
    
    elif re.search(r'financial|investments?|money|laundering|business|tax|schemes?|fraudulent', title, re.IGNORECASE):
        return 'Financial Fraud'
    
    elif re.search(r'drugs?|enforcement|narcotics?|trafficking|abuse|medications?|controlled.*substances?|prescriptions?|pills?', title, re.IGNORECASE):
        return 'Drug Enforcement'
    
    elif re.search(r'bribery|corruption|kickback|payments?|payoffs?|illegal.*government', title, re.IGNORECASE):
        return 'Bribery/Corruption'
    
    else:
        return 'Other'

filtered_df2['Topic'] = filtered_df2.apply(lambda x: classify_topic(x['Title']), axis=1)

In [ ]:
# new df to group by topics and counts
topics_counts = filtered_df2[df2021['Category'] == 'Criminal and Civil Actions'].groupby(['Year-Month', 'Topic']).size().reset_index(name='Count')

In [ ]:
# make the plot
alt.Chart(topics_counts).mark_line().encode(
    x=alt.X('Year-Month:T', title='Month', 
            axis=alt.Axis(
                format='%b %Y',  
                tickCount=12,  
                labelAngle=-45, 
                tickSize=5      
            )),
    y=alt.Y('Count:Q', title='Number of Enforcement Actions'),
    color=alt.Color('Topic:N', title='Topic'),
).properties(
    title='Enforcement Actions by Topic in Criminal and Civil Actions',
    width=800 
)

Source: https://pynative.com/python-regex-search/

## Step 4: Create maps of enforcement activity

### 1. Map by State (PARTNER 1)


In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np

states_gdf = gpd.read_file("data/cb_2018_us_state_5m/cb_2018_us_state_5m.shp")

In [ ]:
# Extract state names from the Agency column
def extract_state(agency):
    if isinstance(agency, str): 
        match = re.search(r'(?:State) of([A-Za-z\s]+)', agency)
        return match.group(1).strip() if match else None
    return None

# Create a new column with extracted state names
df2021['State'] = df2021['Agency'].apply(extract_state)

# Filter out rows where 'State' is missing
df2021_filtered = df2021.dropna(subset=['State'])

In [ ]:
# Group by state and count enforcement actions
actions_by_state = df2021_filtered.groupby('State').size().reset_index(name='Enforcement Actions')

# Make sure column names match
actions_by_state = actions_by_state.rename(columns={"State": "NAME"})

In [ ]:
# Merge the data
states_gdf = states_gdf.merge(actions_by_state, on='NAME', how='left')

# Filter out non-continental US States
states_gdf = states_gdf[~states_gdf['NAME'].isin(['Alaska', 'Hawaii', 'Puerto Rico', 'American Samoa', 'United States Virgin Islands', 'Guam', 'Commonwealth of the Northern Mariana Islands'])]

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))
states_gdf.plot(column='Enforcement Actions', cmap='OrRd',
                linewidth=0.1, edgecolor='0.8', legend=True, ax=ax)

states_gdf.boundary.plot(ax=ax, color='black', linewidth=0.8, alpha=0.7)

ax.set_title("Number of Enforcement Actions by State", fontsize=16)
ax.axis('off')

plt.tight_layout()
plt.show()

### 2. Map by District (PARTNER 2)


In [ ]:
district_data_2021 = df2021[df2021["Agency"].str.contains("District", case=False, na=False)]

district_data_2021["Cleaned_District"] = (
    district_data_2021["Agency"]
    .str.replace(r"U\.?S\.?\s+District\s+|Office of the|District of", "", regex=True)
    .str.strip()
)

district_counts = district_data_2021.groupby("Cleaned_District").size().reset_index(name="Enforcement_Count")

district_shapefile = 

merged_data = district_shapefile.merge(district_counts, left_on="Judicial_District", right_on="Cleaned_District", how="left")

fig, ax = plt.subplots(1, 1, figsize=(12, 8))
merged_data.plot(
    column="Enforcement_Count",
    cmap="OrRd",
    linewidth=0.8,
    edgecolor="0.8",
    legend=True,
    ax=ax
)
ax.set_title("Enforcement Actions by US Attorney District (2021)")
plt.show()

## Extra Credit

### 1. Merge zip code shapefile with population

### 2. Conduct spatial join

### 3. Map the action ratio in each district